## Packages

From termainal I did:

```
pipenv install
pipenv install jupyter pymysql sqlalchemy requests
```


In [1]:
import pymysql.cursors
import requests
from datetime import datetime
from sqlalchemy import create_engine
import hashlib 
import os
import json
import time
from collections import defaultdict
from flask import jsonify
import re
import pandas as pd
from gspread_pandas import Spread, Client

# variables we'll need
host = os.environ['DBHOST']
port = 3306
dbname = "collaborate"
user = os.environ['DBUSER']
password = os.environ['DBPASSWORD']

In [2]:
#### Main function ####

def handler(incoming):
    
    ## Put code here
    
    
    return True

In [3]:
########

## Get list of projects

In [4]:
main_spread = Spread('1wZDpHfIqKBEhmS_F485kFKmrBUAbrfMqu2HW4NOY6BE')

In [5]:
project_list = main_spread.sheet_to_df(index=0)

In [6]:
project_list

,friendly_name,dialogflow_project_id,google_spreadsheet_id
0,Fruit Test,propublicafruittest-pyrata,1uHvY_Z0lpGdvAgkfTt-sNXMe5yGOiq0NZJju0aLej1E


In [7]:
# create sqlalchemy engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                   .format(user=user,
                           pw=password,
                           host=host,
                           db=dbname))

In [8]:
# this will become a loop of projects pulled from a 
# master google spreadsheet
project = "propublicafruittest-pyrata"
spreadsheet_id = "1uHvY_Z0lpGdvAgkfTt-sNXMe5yGOiq0NZJju0aLej1E"

In [9]:
# Get the data we need

# open the database connection
dbConnection    = engine.connect()

df = pd.read_sql(f"SELECT `identifier`, `item_key`, `item_value` FROM `data_pieces` WHERE `project` = '{project}'", dbConnection)
columns_df = pd.read_sql(f"SELECT * FROM `column_tracker` WHERE `project` = '{project}'", dbConnection)
text_df = pd.read_sql(f"SELECT * FROM `text_log` WHERE `project` = '{project}'", dbConnection)
first_contact_df = pd.read_sql(f"SELECT * FROM `first_contact` WHERE `project` = '{project}'", dbConnection)

dbConnection.close()

In [10]:
pivoted = df.pivot(index='identifier', columns='item_key', values=['item_value'])

In [11]:
pivoted

item_value                            \
item_key                  Default Welcome Intent Get City Get Fruit  fruit   
identifier                                                                   
test-session-7e9945fa1a0a                   Hola      NaN       NaN    NaN   
test-session-f7dc5a63848f                  hello  Madison    Grapes  Grape   

                                    
item_key                  geo-city  
identifier                          
test-session-7e9945fa1a0a      NaN  
test-session-f7dc5a63848f  Madison

In [12]:
pivoted.columns = pivoted.columns.get_level_values(1)
pivoted.reset_index(inplace=True) 

In [13]:
pivoted

item_key,identifier,Default Welcome Intent,Get City,Get Fruit,fruit,geo-city
0,test-session-7e9945fa1a0a,Hola,NaN,NaN,NaN,NaN
1,test-session-f7dc5a63848f,hello,Madison,Grapes,Grape,Madison


In [14]:
pivoted.columns

Index(['identifier', 'Default Welcome Intent', 'Get City', 'Get Fruit',
       'fruit', 'geo-city'],
      dtype='object', name='item_key')

In [15]:
columns_df.sort_values(by=['created_at'], inplace=True)

In [16]:
ordered_columns = columns_df['col'].tolist()

In [17]:
ordered_columns.insert(0, "identifier")

In [18]:
ordered_columns

['identifier',
 'Default Welcome Intent',
 'geo-city',
 'Get City',
 'fruit',
 'Get Fruit']

In [19]:
pivoted_ordered = pivoted[ordered_columns]

In [20]:
pivoted_ordered

item_key,identifier,Default Welcome Intent,geo-city,Get City,fruit,Get Fruit
0,test-session-7e9945fa1a0a,Hola,NaN,NaN,NaN,NaN
1,test-session-f7dc5a63848f,hello,Madison,Madison,Grape,Grapes


In [21]:
text_df

,id,created_at,project,identifier,raw_text
0,1,2020-07-14 11:46:36,propublicafruittest-pyrata,test-session-7e9945fa1a0a,Hola
1,2,2020-07-14 17:50:11,propublicafruittest-pyrata,test-session-f7dc5a63848f,hello
2,3,2020-07-14 17:50:34,propublicafruittest-pyrata,test-session-f7dc5a63848f,Madison
3,4,2020-07-14 17:50:40,propublicafruittest-pyrata,test-session-f7dc5a63848f,Grapes


In [22]:
text_concat = text_df.groupby(['identifier'])['raw_text'].apply(' | '.join).reset_index()



In [23]:
text_concat

,identifier,raw_text
0,test-session-7e9945fa1a0a,Hola
1,test-session-f7dc5a63848f,hello | Madison | Grapes


In [24]:
merge1 = pd.merge(pivoted_ordered, text_concat, on="identifier")

In [25]:
merge1

,identifier,Default Welcome Intent,geo-city,Get City,fruit,Get Fruit,raw_text
0,test-session-7e9945fa1a0a,Hola,NaN,NaN,NaN,NaN,Hola
1,test-session-f7dc5a63848f,hello,Madison,Madison,Grape,Grapes,hello | Madison | Grapes


In [83]:
first_contact_df.sort_values(by=['created_at'], inplace=True)

In [84]:
final_table = pd.merge(first_contact_df, merge1, on="identifier")

In [85]:
final_table

,proj_ident_hash,project,identifier,created_at,Default Welcome Intent,geo-city,Get City,fruit,Get Fruit,raw_text
0,dd8b594c008ad32f17f60f6a857f9e9e,propublicafruittest-pyrata,test-session-7e9945fa1a0a,2020-07-14 11:46:36,Hola,NaN,NaN,NaN,NaN,Hola
1,bbb05269fc4f167a29ace916cbf1b30e,propublicafruittest-pyrata,test-session-f7dc5a63848f,2020-07-14 17:50:11,hello,Madison,Madison,Grape,Grapes,hello | Madison | Grapes


In [86]:
# drop the project columns for the spreadsheet
final_table.drop(columns=['proj_ident_hash', 'project'], inplace = True)

In [87]:
# final_table

## Trying a service account

In [88]:
spread = Spread(spreadsheet_id)

In [89]:
spread.df_to_sheet(final_table, index=False, sheet='Sheet1', start='A1', replace=True)